In [10]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px

In [112]:
import sys
from io import StringIO
import numpy as np
import pandas as pd

# TODO: 
# - Store results in csv if optioned
# - Add option for directory

### DAT FILES
def extract_table(contents: str, keyword: str, delim: str ='\n\n\n') -> str:
    """
    Extracts strings consisting of only table values specific for .dat files
    """
    # Find keyword index
    start_index = contents.find(keyword)
    if start_index == -1:
        raise ValueError(f"Keyword '{keyword}' not found in file.")
    
    start_index += len(keyword)
    
    # Start remaining text after keyword
    remaining_text = contents[start_index:]
    
    # Find 2 occurrences of delim
    positions = []
    search_start = 0
    
    count = 0
    while count != 2:
        pos = remaining_text.find(delim, search_start)
        if pos == -1:
            break
        if count == 0:
            table_start_index = pos + 3
        elif count == 1:
            table_end_index = pos
            
        count += 1
        search_start = pos +3
    
    try:
        extracted_text = remaining_text[table_start_index:table_end_index]
    except:
        extracted_text = remaining_text[table_start_index:]
    
    return extracted_text

def get_mode_table_df(contents: str):
    keyword = 'E I G E N V A L U E    O U T P U T'
    table = extract_table(contents, keyword)

    mode_table_df = pd.read_csv(
        StringIO(table),
        sep=r'\s+',
        header=None,
        names=[
            'mode_no',
            'eigenvalue',
            'freq (rad/time)',
            'freq (cycles/time)',
            'generalized mass',
            'composite model damping'
        ]
    )

    mode_table_df = mode_table_df[['mode_no', 'freq (cycles/time)']].copy()
    mode_table_df.rename(columns = {'freq (cycles/time)': 'freq'}, inplace=True)

    return mode_table_df

def get_mode_df(contents, mode_number):
    num_str = str(mode_number)
    keyword = f"E I G E N V A L U E    N U M B E R{num_str.rjust(6, ' ')}"
    table_content_start = contents.find(keyword)
    table_content = contents[table_content_start + len(keyword):]

    table = extract_table(table_content, 'U3', delim='\n  \n')

    max_index = table.find('MAX') 
    if max_index == -1:
        raise Exception("Incorrect file format")
    table = table[:max_index].rstrip()

    # print(table)
    mode_df = pd.read_csv(
        StringIO(table),
        sep=r'\s+',
        header=None,
        names=[
            'node_no',
            'U1',
            'U2',
            'U3'
        ]
    )

    return mode_df    


### INP FILES
def extract_inp_table(inp_contents: str, start_keyword: str, end_keyword: str):
    start_index = inp_contents.find(start_keyword)
    end_index = inp_contents.find(end_keyword)

    start_index += len(start_keyword) + 1
    extracted_text = inp_contents[start_index:end_index]

    return extracted_text

def get_node_df(inp_contents: str): # from inp file
    start_keyword = "*NODE"
    end_keyword = "**HWCOLOR COMP"
    table = extract_inp_table(inp_contents, start_keyword, end_keyword)

    node_table_df = pd.read_csv(
        StringIO(table),
        sep=r'[,\s]+',
        header=None,
        names=[
            'node_no',
            'x',
            'y',
            'z'
        ]
    )

    return node_table_df

def get_proportions(mode_df, n):
    # only use positive y values
    mode_df = mode_df[mode_df['U2'] >= 0].copy()
    
    mode_df['sq_x'] = mode_df['U1']**2
    mode_df['sq_y'] = mode_df['U2']**2
    mode_df['sq_z'] = mode_df['U3']**2
    
    sumsq_x = mode_df['sq_x'].sum()
    sumsq_y = mode_df['sq_y'].sum()
    sumsq_z = mode_df['sq_z'].sum()

    total_energy = sumsq_x + sumsq_y + sumsq_z

    oop = (sumsq_y) / total_energy
    ip = (sumsq_x + sumsq_z) / total_energy

    # print(f"Mode {n} has OOP: {oop*100}%, IP: {ip*100}%.")
    return oop*100, ip*100, sumsq_x, sumsq_y, sumsq_z

def get_average_disp(mode_df, n):
    return mode_df['resultant_xyz'].sum() / len(mode_df)

In [74]:
def contains_node(mode_df, threshold=7, lower_p_thres=0):
    mode_df['resultant'] = np.sqrt(mode_df['U1']**2 + mode_df['U2']**2 + mode_df['U3']**2)
    zero_proportion = (mode_df['resultant'] < threshold).sum()/len(mode_df)
    return zero_proportion > lower_p_thres

In [12]:
dat_file = "data/C346RS_frnt_rotor_modal_separation_10Jun25.dat"
inp_file = "data/C346RS_frnt_rotor_modal_separation_10Jun25.inp"

In [13]:
with open(dat_file, "r") as file:
    contents = file.read()

In [14]:
with open(inp_file, "r") as inp_file:
    inp_contents = inp_file.read()

In [15]:
outplane_modes = [29, 36, 45, 48, 65, 71, 94, 103]
inplane_modes = [32, 33, 46, 47, 68, 69, 99, 100]

In [16]:
node_df = get_node_df(inp_contents)

C:\Users\BHENDRAT\AppData\Local\Temp\ipykernel_70264\2966383457.py:116: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  node_table_df = pd.read_csv(


In [105]:
def plot_sum_graph(mode_no):
    # TODO: Obtain node_df first
    mode_df = get_mode_df(contents, mode_no)
    mode_df['resultant'] = np.sqrt(mode_df['U1']**2 + mode_df['U2']**2 + mode_df['U3']**2)
    sum_df = mode_df[['node_no','resultant']] # node and sum
    merged_df = pd.merge(node_df, sum_df, on='node_no', how='inner')
    df = merged_df
    
    custom_colorscale = [
        [0.0, '#00008B'],  # dark blue
        [0.5, '#FFFF00'],  # yellow
        [1.0, '#FF0000'],  # red
    ]

    min_val = min(df['x'].min(), df['y'].min(), df['z'].min())
    max_val = max(df['x'].max(), df['y'].max(), df['z'].max())
    
    fig = px.scatter_3d(
        df,
        x='x',
        y='y',
        z='z',
        color='resultant',
        color_continuous_scale=custom_colorscale,
        size_max=10,
        hover_data=['node_no', 'resultant']
    )
    
    fig.update_layout(
        title=f"Mode number ({mode_no})",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            xaxis=dict(range=[min_val, max_val]),
            yaxis=dict(range=[min_val, max_val]),
            zaxis=dict(range=[min_val, max_val]),
        )
    )
    
    fig.show(renderer='browser')

In [32]:
def plot_graph(df):
    # Calculate the combined min and max over all three axes
    min_val = min(df['x'].min(), df['y'].min(), df['z'].min())
    max_val = max(df['x'].max(), df['y'].max(), df['z'].max())
    custom_colorscale = [
        [0.0, '#00008B'],  # dark blue
        [0.5, '#FFFF00'],  # yellow
        [1.0, '#FF0000'],  # red
    ]   
    fig = px.scatter_3d(
        df,
        x='x',
        y='y',
        z='z',
        size_max=2,
        color='x',
        color_continuous_scale=custom_colorscale,
        hover_data=['node_no']
    )


    
    fig.update_layout(
        title='Interactive 3D Scatter Plot Colored by Magnitude of sum',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            xaxis=dict(range=[min_val, max_val]),
            yaxis=dict(range=[min_val, max_val]),
            zaxis=dict(range=[min_val, max_val]),
        )
    )
    
    fig.show(renderer='browser')

In [33]:
plot_graph(node_df)

In [104]:
plot_sum_graph(98)

In [113]:
inplane_preds = []
for mode_number in range(1,132):
    mode_df = get_mode_df(contents, mode_number)
    oop, ip, x, y, z = get_proportions(mode_df, mode_number)

    if ip > 96 and (x+z) > 100000:
        if contains_node(mode_df):
            inplane_preds.append(mode_number)

In [77]:
inplane_preds

[32, 33, 46, 47, 68, 69, 100, 101]

Catching out of planes

In [115]:
for i in range(1, 132):
    mode_df = get_mode_df(contents, i)
    oop, ip, x, y, z = get_proportions(mode_df, i)
    print(i, oop)

1 97.55958524998591
2 97.55958867759088
3 99.4802581833743
4 97.90446372556245
5 97.90447879408939
6 1.4835888910720667e-06
7 17.052412038016946
8 17.116658003327892
9 97.72889480833173
10 97.72195282139336
11 8.979240841200822
12 8.974101953169585
13 89.26918764844231
14 97.16850542555535
15 97.15967610944371
16 84.04405420179867
17 82.18111162301341
18 83.79947534338307
19 83.80050937197329
20 97.16489512955727
21 97.22952274100692
22 10.490359160038345
23 10.48984777831746
24 7.464204119851519
25 56.962265004853165
26 84.74237313135278
27 84.74064796778212
28 97.04059974943937
29 97.04082615701067
30 35.047385274654026
31 31.15292476009038
32 1.8409391402678037
33 2.848038240312679
34 50.809220163437715
35 50.81101256016153
36 96.85327508250947
37 96.8532773237032
38 55.32892035899953
39 55.3408448667558
40 71.32245170383612
41 71.1881432250378
42 66.21982313039544
43 66.3324014240586
44 96.63286497283377
45 96.64586486952234
46 0.28582327353845216
47 0.3149477128587458
48 96.403169

In [ ]:
# To consider
# - A

In [127]:
def search_oop(contents, start, reverse=False, max_modes = 136):
    """
    Linear check
    """
    checks = 0 # if checks > 5, then take largest
    if reverse:
        inc = -1
    else:
        inc = 1

    oop_values = []
    while (checks <= 5):
        checks += 1
        start += inc
        if start == 0 or start == max_modes:
            break

        mode_df = get_mode_df(contents, start)
        oop, _, _, _, _ = get_proportions(mode_df, start)

        if oop > 95:
            return start
        else:
            oop_values.append((start, oop))

    largest_oop = max(oop_values, key=lambda x: x[1])
    return largest_oop[0]

In [128]:
search_oop(contents, 100, reverse=False)

105